In [12]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import torch
import torchaudio as ta
import torchaudio.functional as taf
import torchaudio.transforms as tat

print(torch.__version__)
print(ta.__version__)

import matplotlib
import matplotlib.pyplot as plt
from IPython.display import Audio, display

import pandas as pd
import os
import pprint

from typing import *
import itertools
from collections import Counter

import numpy as np
from datetime import datetime

1.8.1
0.8.0a0+e4e171a


In [2]:
from torch.utils.data import Dataset, DataLoader
from dataclasses import dataclass
import glob
import random
from torch import nn
import torch.nn.functional as F
from catalyst import dl, utils
from catalyst.data.transforms import ToTensor

In [3]:
from lark.learner import *
from lark.Cnn14_DecisionLevelAtt import *

In [4]:
cfg = Config(site='SSW')

In [5]:
cfg.as_dict()

{'site': 'SSW',
 'data_dir': 'data/birdclef-2021',
 'bs': 32,
 'n_workers': 12,
 'training_dataset_size': 3840,
 'duration': 5,
 'sr': 32000,
 'n_frames': 160000,
 'n_fft': 512,
 'window_length': 512,
 'n_mels': 64,
 'hop_length': 312,
 'f_min': 150,
 'f_max': 15000,
 'labels': ['amecro',
  'amegfi',
  'amerob',
  'balori',
  'belkin1',
  'bkcchi',
  'blujay',
  'cangoo',
  'chswar',
  'comgra',
  'comyel',
  'dowwoo',
  'eastow',
  'eawpew',
  'gockin',
  'grycat',
  'haiwoo',
  'norcar',
  'norfli',
  'norwat',
  'ovenbi1',
  'rebwoo',
  'reevir1',
  'rewbla',
  'sonspa',
  'swaspa',
  'woothr',
  'yebsap']}

In [6]:
vs = ValidDataset(cfg)

In [7]:
vdl = vs.loader

In [8]:
ts = TrainDataset(cfg)

In [9]:
tdl = ts.loader

In [10]:
model = Cnn14_DecisionLevelAtt.load(cfg)

In [13]:
exp = Experiment(cfg)
exp.init(f"pann {datetime.now()}")


psutil is not installed. You will not be able to abort this experiment from the UI.
psutil is not installed. Hardware metrics will not be collected.


https://ui.neptune.ai/botkop/lark/e/LAR-3


In [67]:
lr = 1e-4
n_epochs = 10
n_samples = len(tdl)

loss_fn = PANNsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=lr*10, steps_per_epoch=n_samples, epochs=n_epochs)

In [68]:
model = model.cuda()
loss_fn = loss_fn.cuda()

In [69]:
# from tqdm import tqdm
import sys
from tqdm.notebook import tqdm

def tv_loop(dl, mode):
    if mode == 'train':
        model.train()
    else:
        model.eval()
        
    def inner_loop():
        nonlocal tot_loss
        nonlocal tot_score
        for x, y in pbar:
            x = x.cuda()
            y = y.cuda()
            pred = model(x)
            loss = loss_fn(pred, y)
            score = f1(y, pred, 0.5)
            
            pbar.set_description(f"{mode} loss: {loss:>8f} f1: {score:>8f}")
            neptune.log_metric(f'batch_{mode}_loss', loss)
            neptune.log_metric(f'batch_{mode}_f1', score)

            tot_loss += loss.item()
            tot_score += score
            if mode == 'train':
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                scheduler.step()
        
    tot_loss = 0
    tot_score = 0
       
    with tqdm(dl, leave=False) as pbar:
        if mode == 'valid':
            with torch.no_grad():
                inner_loop()
        else:
            inner_loop()
            
    n_batches = len(dl)
    tot_loss /= n_batches
    tot_score /= n_batches
    return tot_loss, tot_score

In [70]:
with tqdm(range(n_epochs)) as pbar:
    for epoch in pbar:
        train_loss, train_score = tv_loop(tdl, 'train')
        valid_loss, valid_score = tv_loop(vdl, 'valid')
        msg = f"epoch: {epoch + 1:3d} train loss: {train_loss:>8f} train f1: {train_score:>8f} valid loss: {valid_loss:>8f} valid f1: {valid_score:>8f}"
        print(msg)
        neptune.log_metric(f'epoch_train_loss', train_loss)
        neptune.log_metric(f'epoch_valid_loss', valid_loss)
        neptune.log_metric(f'epoch_train_f1', train_score)
        neptune.log_metric(f'epoch_valid_f1', valid_score)


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/120 [00:00<?, ?it/s]

  0%|          | 0/38 [00:00<?, ?it/s]

epoch:   1 train loss: 0.542606 train f1: 0.034940 valid loss: 0.118420 valid f1: 0.210526


  0%|          | 0/120 [00:00<?, ?it/s]

...

  0%|          | 0/38 [00:00<?, ?it/s]

epoch:   9 train loss: 0.057092 train f1: 0.719075 valid loss: 0.121524 valid f1: 0.260631


  0%|          | 0/120 [00:00<?, ?it/s]

  0%|          | 0/38 [00:00<?, ?it/s]

epoch:  10 train loss: 0.058275 train f1: 0.710886 valid loss: 0.125516 valid f1: 0.258484
